In [ ]:
import os
import ast
import json
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
exp_dir = Path("/Users/andreas/workspace/thesis-code/out/fedless-mnist-flower")
#session_id = "d302c4be-cf23-4ce2-8a6a-c0975870fcf4"

#Fedless Shakespeare 3c1dd162-a3b9-483a-b7da-faf5aa0a800d
#Fedless MNIST cc1ee881-e81e-46fe-81e9-87872de160f4
timing_dfs = []
clients_dfs = []
for f in exp_dir.glob("timing_*.csv"):
    timing_dfs.append(pd.read_csv(exp_dir / f, index_col="round_id"))
for f in exp_dir.glob("clients_*.csv"):
    clients_dfs.append(pd.read_csv(exp_dir / f))
timing_df = pd.concat(timing_dfs)
clients_df = pd.concat(clients_dfs)
#timing_df = pd.read_csv(exp_dir / f"timing_{session_id}.csv")
#clients_df = pd.read_csv(exp_dir / f"clients_{session_id}.csv")

clients_df["function"] = clients_df["function"].map(json.loads)

In [ ]:
#timing_df['client_accs'] = timing_df['all_accuracy'].map(ast.literal_eval)#.map(lambda li: [float(l) for l in li])
#timing_df = timing_df.explode('client_accs', ignore_index=True)
#timing_df['client_accs'] = timing_df['client_accs'].astype(float)
#timing_df['client_accs'] = timing_df['client_accs'].astype(float)

In [ ]:
fig, ax = plt.subplots()
sns.lineplot(x="round_id", y="global_test_accuracy", data=timing_df, ax=ax, label="Weighted Mean")
#sns.lineplot(x="round_id", y="median_test_accuracy", data=timing_df, ax=ax, label="Weighted Mean")
#sns.lineplot(x="round_id", y="client_accs", data=timing_df, ax=ax, label="Client Distribution")
#sns.lineplot(x="round_id", y="seconds", data=clients_df, ax=ax)

In [ ]:
sns.set_theme(style="whitegrid")
fig, ax = plt.subplots(figsize=(5, 5))
#sns.set_color_codes("muted")
#sns.set_color_codes("muted")
#sns.barplot(x="round_id", y="round_seconds", data=timing_df, ax=ax, label="Weighted Mean", color="b")
#sns.barplot(x="round_id", y="clients_finished_seconds", data=timing_df, ax=ax, label="Total", color="b")
#sns.barplot(x="round_id", y="clients_finished_seconds", data=timing_df, ax=ax, label="Clients", color="r")
#sns.barplot(x="round_id", y="aggregator_seconds", data=timing_df, ax=ax, label="Aggregator", color="g")
#timing_df[["clients_finished_seconds", "aggregator_seconds", "evaluator_seconds"]].plot.bar(stacked=True, ax=ax)
cold_starts = timing_df[timing_df.index == 0]
warm_starts = timing_df[timing_df.index != 0]
cold_starts_mean = cold_starts.mean(numeric_only=True).to_frame().T
warm_starts_mean = warm_starts.mean(numeric_only=True).to_frame().T
cold_starts_std = cold_starts.std(numeric_only=True).to_frame().T
warm_starts_std = warm_starts.std(numeric_only=True).to_frame().T
all_df = pd.concat((cold_starts_mean, warm_starts_mean))
all_df[["clients_finished_seconds", "aggregator_seconds"]].plot.bar(stacked=True, ax=ax)
ax.set_xticklabels(["Cold", "Warm"], rotation=0)
#warm_starts_mean[["clients_finished_seconds", "aggregator_seconds", "evaluator_seconds"]].plot.bar(stacked=True, ax=ax)
#timing_df[["clients_finished_seconds", "aggregator_seconds", "evaluator_seconds"]].plot.bar(stacked=True, ax=ax)
#timing_df[["clients_finished_seconds", "aggregator_seconds", "evaluator_seconds"]].mean().plot.pie(subplots=True, ax=ax)

In [ ]:
warm_starts_mean
#fig, ax = plt.subplots()
#sns.barplot(x=timing_df.index, y="round_seconds", data=timing_df, ax=ax, color="b")

In [ ]:
fig, ax = plt.subplots()
sns.lineplot(x="round_id", y="round_seconds", data=timing_df, ax=ax, color="b")

In [ ]:
sns.set_style('whitegrid')
fig, ax = plt.subplots()
df = timing_df.copy()
df["time"] = df["round_seconds"].cumsum() / 3600
sns.lineplot(x="round_id", y="time", data=df, ax=ax)
ax.set_ylabel("Time (h)")
ax.set_xlabel("Round")

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
sns.boxplot(x="client_id", y="seconds", data=clients_df, ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
sns.boxplot(y="seconds", data=clients_df, showfliers=False, showbox=False, whis=[1.0,99.0], ax=ax)
sns.violinplot(y="seconds", data=clients_df, ax=ax)
fig.savefig("/Users/andreas/desktop/dist-violin.pdf")

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
sns.boxplot(y="seconds", data=clients_df, ax=ax)
fig.savefig("/Users/andreas/desktop/dist-box.pdf")

In [ ]:
exp_dir = Path("/Users/andreas/workspace/thesis-code/out/fedkeeper-comp/")
fig, ax = plt.subplots()
non_ldp_df = pd.read_csv(
    exp_dir / "fedkeeper_mnist_100_25_5_0.99_non_ldp" / f"timing_ab622373-4075-4bd2-b97d-4cf00a79440a.csv")
ldp_df = pd.read_csv(
    exp_dir / "fedkeeper_mnist_100_25_5_0.99_ldp" / f"timing_6d725adc-7e03-41d8-94a4-c135fd613adb.csv")
#sns.lineplot(x="round_id", y="global_test_accuracy",
#             data=non_ldp_df,
#             ax=ax,
#             label="Non LDP")
#sns.lineplot(x="round_id", y="global_test_accuracy",
#             data=ldp_df,
#             ax=ax,
#             label="LDP")
sns.lineplot(x="round_id", y="clients_finished_seconds",
             data=non_ldp_df,
             ax=ax,
             label="Non LDP")
sns.lineplot(x="round_id", y="clients_finished_seconds",
             data=ldp_df,
             ax=ax,
             label="LDP")

In [ ]:
#
clients_df["function_type"] = clients_df["function"].map(lambda x: x["type"])
clients_df["function_host"] = clients_df["function"].map(lambda x: x["params"]["url"].split("function")[0])

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
sns.boxplot(x="function_host", y="seconds", data=clients_df, ax=ax)
plt.xticks(rotation=45)
fig.savefig("/Users/andreas/Desktop/time.pdf")


In [ ]:
timing_df